## Quantifying a Spectrum

#### Note:  This notebook uses some archaic syntax that is not necessary if the `reference` and `references` functions are used to build the fitting standards.

This script show how to pull together the functionality in the NeXL X-ray microanalysis library to quantify a electron-excited X-ray spectrum using standard spectra.

First, we need to load the necessary libraries - `NeXLSpectrum` and `NeXLMatrixCorrection` from NeXL, `Gadfly` for plotting and `DataFrames` for tabulation. 

In [ ]:
using NeXLSpectrum
using NeXLMatrixCorrection
using Gadfly
using DataFrames

Next we specify where the spectra are located and load the spectrum from a file.  `loadspectrum(...)` can sniff the format of a spectrum file and load 'ISO/EMSA', 'ASPEX TIFF' and 'Bruker SPX' files automatically.

Once the spectrum is loaded into `unk`, we use the data in `unk` to build a suitable detector with a FWHM at Mn K$\alpha$ of 132.0 eV and a low-level discriminator cut-off of 120 channels.  Finally, we set the `:Detector` property of the spectrum to `det`.

Finally, we use `Gadfly`, which `NeXLSpectrum` has specialized to understand `Spectrum` structures, to plot the spectrum with KLM markers from 0.0 eV to 8,000 eV.

In [ ]:
path = "K309"
unk = loadspectrum(joinpath(path,"K309.msa"))
det = matching(unk, 132.0, 120)
unk[:Detector] = det
set_default_plot_size(10inch,2.8inch)
plot(unk, klms=[n"O", n"Al", n"Si", n"Ca", n"Fe", n"Ba"], xmax=8.0e3)

Next, we define and load the reference spectra.  Here we use `n".."` notation to parse the element symbol and `mat"..."` to parse the composition.

In [ ]:
refs = ( 
    ( n"O", "Al2O3 std.msa", mat"Al2O3" ),
    ( n"Al", "Al2O3 std.msa", mat"Al2O3" ),
    ( n"Si", "Si std.msa", mat"Si" ),
    ( n"Ca", "CaF2 std.msa", mat"CaF2" ),
    ( n"Fe", "Fe std.msa", mat"Fe" ),
    ( n"Ba", "BaF2 std.msa", mat"BaF2" )
)
refspecs = Dict()
for (elm, file, comp) in refs
    spec = loadspectrum(joinpath(path,file))
    spec[:Detector] = det
    spec[:Composition] = comp
    refspecs[elm] = spec
end
plot(values(refspecs)..., autoklms=true, xmax=8.0e3, norm=ScaleDose())

To perform a top-hat filter-fit, we must first filter the reference spectra.  Each spectrum can contribute one or more `Reference[..]` datums depending upon how many contiguous regions of channels are represented by the element for which the spectrum is a reference.

`buildfilter(...)` builds the matrix which represents the top-hat filter.

`filterreference(...)` applies the filter to the reference spectrum. In this case it creates 8 `Reference[...]` structures for 6 elements.

In [ ]:
filt = buildfilter(det)
frs = mapreduce(elm->filterreference(filt, refspecs[elm], elm, refspecs[elm][:Composition]), append!, keys(refspecs))

Finally, we apply the filter to the unknown and fit the filtered references.  The `false` says that negative k-ratios should not be set to zero and the fit repeated with that element removed.

The results are tabulated as a `DataFrame`

In [ ]:
ENV["LINES"] = 60; ENV["COLUMNS"]=120
res=fit_spectrum(unk, filt, frs, false)
asa(DataFrame,res)

To visualize these results, `Gadfly` has been specialized to plot `FilterFitResult` objects like this.  The plot shows the range of channels involved in each fitted ROI along with the associated k-ratio.  The red line is the filter fit residual.  It shows which x-rays were not accounted for as characteristic X-rays in one of the references.  You can use the residual to determine if any elements have been overlooked.

You can see that some of the ROIs are narrow (like the O K) while others are broad (like the Ba L). You can also see which ROIs interfere with each other (like the O K and Fe L or the Al K and Si K).

In [ ]:
plot(res)

The final step is to take the k-ratios and use iteration to estimate the composition.

In [ ]:
ENV["LINES"] = 60; ENV["COLUMNS"]=200
iterres = quantify(res, mc=XPP)
asa(DataFrame, iterres)

In [ ]:
asa(DataFrame, [ iterres.comp ])